# Prepare

## Load Dataset and Packages

In [ ]:
from datetime import datetime
import os
from glob import glob
import shutil
import numpy as np
import random
import math

from PIL import Image
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from keras.models import model_from_json
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [ ]:
num_classes = 13
input_shape = (600, 800, 3)

drop_out = 0.5
decay_steps = 5
decay_rate = 0.8
learning_rate = 0.001
batch_size = 2
num_epochs = 1

## Build Model

In [ ]:
def modified_unet(input_shape):
    bn=True
    
    ki= 'he_uniform'  # 'glorot_normal' "he_normal"
    
    concat_axis = -1
    
    inputs = layers.Input(input_shape)
    
    conv1 = layers.Conv2D(64, (5,5), padding="same", activation="relu", kernel_initializer=ki)(inputs)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Dropout(drop_out)(conv1)
    conv1 = layers.Conv2D(64, (5,5), padding="same", activation="relu", kernel_initializer=ki)(conv1)
    conv1 = layers.BatchNormalization()(conv1)
    
    pool1 = layers.MaxPooling2D(pool_size=(2,2))(conv1)
    
    conv2 = layers.Conv2D(96, (3,3), padding="same", activation="relu", kernel_initializer=ki)(pool1)  # 96
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Dropout(drop_out)(conv2)
    conv2 = layers.Conv2D(96, (3,3), padding="same", activation="relu", kernel_initializer=ki)(conv2)
    conv2 = layers.BatchNormalization()(conv2)
    
    pool2 = layers.MaxPooling2D(pool_size=(2,2))(conv2)
    
    conv3 = layers.Conv2D(128, (3,3), padding="same", activation="relu", kernel_initializer=ki)(pool2)  # 128
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Dropout(drop_out)(conv3)
    conv3 = layers.Conv2D(128, (3,3), padding="same", activation="relu", kernel_initializer=ki)(conv3)
    conv3 = layers.BatchNormalization()(conv3)
    
    pool3 = layers.MaxPooling2D(pool_size=(2,2))(conv3)
    
    conv4 = layers.Conv2D(256, (3,3), padding="same", activation="relu", kernel_initializer=ki)(pool3)  # 256
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.Dropout(drop_out)(conv4)
    conv4 = layers.Conv2D(256, (3,3), padding="same", activation="relu", kernel_initializer=ki)(conv4)
    conv4 = layers.BatchNormalization()(conv4)
    
    #######
    
    conv4 = layers.Conv2D(512, (3,3), dilation_rate=2, padding="same", activation="relu", kernel_initializer=ki)(conv4)  # 512
    cat6 = conv4
    
    conv6 = layers.Conv2D(256, (3,3), padding="same", activation="relu", kernel_initializer=ki)(cat6)  # 256
    conv6 = layers.BatchNormalization()(conv6)
    conv6 = layers.Dropout(drop_out)(conv6)
    conv6 = layers.Conv2D(256, (3,3), padding="same", activation="relu", kernel_initializer=ki)(conv6)
    conv6 = layers.BatchNormalization()(conv6)
    upconv6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', kernel_initializer=ki)(conv6)  # 128
    
    up7 = layers.concatenate([upconv6, conv3], axis=concat_axis)
    
    conv7 = layers.Conv2D(128, (3,3), padding="same", activation="relu", kernel_initializer=ki)(up7)  # 128
    conv7 = layers.BatchNormalization()(conv7)
    conv7 = layers.Dropout(drop_out)(conv7)
    conv7 = layers.Conv2D(128, (3,3), padding="same", activation="relu")(conv7)
    conv7 = layers.BatchNormalization()(conv7)
    upconv7 = layers.Conv2DTranspose(96, (2, 2), strides=(2, 2), padding='same', kernel_initializer=ki)(conv7)  # 96
    
    up8 = layers.concatenate([upconv7, conv2], axis=concat_axis)
    
    conv8 = layers.Conv2D(96, (3,3), padding="same", activation="relu", kernel_initializer=ki)(up8)  # 96
    conv8 = layers.BatchNormalization()(conv8)
    conv8 = layers.Dropout(drop_out)(conv8)
    conv8 = layers.Conv2D(96, (3,3), padding="same", activation="relu", kernel_initializer=ki)(conv8)
    conv8 = layers.BatchNormalization()(conv8)
    upconv8 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', kernel_initializer=ki)(conv8)
    
    up9 = layers.concatenate([upconv8, conv1], axis=concat_axis)
    
    conv9 = layers.Conv2D(64, (3,3), padding="same", activation="relu", kernel_initializer=ki)(up9)
    conv9 = layers.BatchNormalization()(conv9)
    conv9 = layers.Conv2D(64, (3,3), padding="same", activation="relu", kernel_initializer=ki)(conv9)
    conv9 = layers.BatchNormalization()(conv9)

    conv9 = layers.Conv2D(num_classes, (1,1), padding="same", activation="softmax", kernel_initializer=ki)(conv9)
    
    model = tf.keras.Model(inputs=inputs, outputs=conv9)
    return model

# Set for Train

## Set for Optimizer

In [ ]:
def precision(y_true, y_pred):
    axes = tuple(range(1, len(y_pred.shape)-1))
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axes)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axes)
    precision = true_positives / (predicted_positives + K.epsilon())
    
    return K.mean(precision)


def recall(y_true, y_pred):
    axes = tuple(range(1, len(y_pred.shape)-1))
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axes)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axes)
    recall = true_positives / (possible_positives + K.epsilon())
    
    return K.mean(recall)

def dice(y_true, y_pred):
    epsilon=1e-6
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    numerator = 2. * K.sum(y_pred * y_true, axes)
    denominator = K.sum(K.square(y_pred) + K.square(y_true), axes)
    
    return K.mean(numerator / (denominator + epsilon))

In [ ]:
def soft_dice_loss(y_true, y_pred):
    epsilon=1e-6
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    numerator = 2. * K.sum(y_pred * y_true, axes)
    denominator = K.sum(K.square(y_pred) + K.square(y_true), axes)
    
    return 1 - K.mean(numerator / (denominator + epsilon))

In [ ]:
model = modified_unet(input_shape)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate), #, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              loss=soft_dice_loss,
              metrics=['accuracy', recall, precision])

# Train Dataset 준비

In [ ]:
os.listdir('data/lab_classroom/')[:10]

In [ ]:
image_paths = glob('data/lab_classroom/*.jpg')

In [ ]:
label_paths = [path.replace('.jpg', '_seg.png') for path in image_paths]

In [ ]:
# 파일이 제대로 있는지 확인 하는 작업
for path in label_paths:
    if not os.path.isfile(path):
        print(path, 'not exists')

## Train과 Test Set 나누기

In [ ]:
test_rate = 0.5
num_images = len(image_paths)
train_images = image_paths[:int(num_images * test_rate)]
test_images = image_paths[int(num_images * test_rate):]
len(train_images), len(test_images)

## DataSet Load

In [ ]:
lbl_path = label_paths[0]

label = np.array(Image.open(lbl_path))[:,:,0]//10
label_onehot = tf.keras.utils.to_categorical(label, num_classes=num_classes)
label_onehot.shape

In [ ]:
lbl_path = label_paths[0]

def load_data(path):
    image = np.array(Image.open(path))
    image = cv2.resize(image, (input_shape[1], input_shape[0]))
    
    lbl_path = path.replace('.jpg', '_seg.png')
    label = np.array(Image.open(lbl_path))[:,:,0]//10
    # label은 resize할 때 꼭!!! NEARST로 해야한다!!
    label = cv2.resize(label, (input_shape[1], input_shape[0]), interpolation=cv2.INTER_NEAREST)
    label_onehot = tf.keras.utils.to_categorical(label, num_classes=num_classes)
    
    return image, label_onehot

In [ ]:
def batch_dataset(image_paths):
    batch_image = []
    batch_label = []
    for image_path in image_paths:
        image, label = load_data(image_path)
        image = np.expand_dims(image, 0)
        label = np.expand_dims(label, 0)
        
        batch_image.append(image)
        batch_label.append(label)
        
    images = np.concatenate(batch_image, 0) / 255. # rescale
    labels = np.concatenate(batch_label, 0)
    
    return images.astype(np.float32), labels.astype(np.float32)


def train_gen(batch_size):
    while True:
        offset = np.random.randint(0, num_images - batch_size)
        yield batch_dataset(train_images[offset:offset+batch_size])

In [ ]:
history = model.fit_generator(
    train_gen(batch_size),
    epochs=num_epochs,
    steps_per_epoch=100,
    validation_data=batch_dataset(test_images)
)

# Test DataSet

## Check Testset

In [ ]:
test_images, test_labels = batch_dataset(test_images[:batch_size])

predicts = model.predict(testset)

In [ ]:
for i in range(batch_size):
    plt.figure(figsize=(30, 100))
    plt.subplot(1, num_classes, 1)
    result = (np.argmax(predicts, -1) * (255 / (num_classes-1))).astype(np.uint8)
    plt.imshow(result[i, :, :])
    for c in range(1, num_classes+1):
        plt.subplot(1, num_classes, c)
        plt.imshow(predicts[i, :, :, c-1], 'gray')
    plt.show()

In [ ]:
result_num = 4


for pred, test, test_label in zip(predicts[:result_num], test_images[:result_num], test_labels[:result_num]):
    argmaxed = np.where(np.argmax(pred, -1) > 0, 1, 0)
    arg_label = np.where(np.argmax(test_label, -1) > 0, 1, 0)
    plt.figure(figsize=(20, 80))
    plt.subplot(1, 3, 1)
    plt.title('Test Images')
    plt.imshow(test[:,:,0], 'gray')
    plt.subplot(1, 3, 2)
    plt.title('Predictions')
    plt.imshow(test[:,:,0], 'gray')
    plt.imshow(argmaxed, alpha=0.7)
    plt.subplot(1, 3, 3)
    plt.title('Ground Truth')
    plt.imshow(test[:,:,0], 'gray')
    plt.imshow(arg_label, alpha=0.7)
    plt.show()